# A股涨跌幅极端统计工具

## 统计逻辑：
- 上涨：当日涨幅>0
- 下跌：当日涨幅<0
- 平家：当日涨幅=0
- 涨跌停：当日涨幅=10-20
- 停牌：当日无数据

## 统计范围
全市场

## 参数设置和操作指南
- 代码列表中，修改开始和结束时间
- 其他不要动，直接执行即可

In [34]:
# 本代码由可视化策略环境自动生成 2022年4月26日 15:09
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


m1 = M.instruments.v2(
    start_date='2022-01-01',
    end_date='2022-04-26',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.input_features.v1(
    features="""price_limit_status_0
return_0"""
)

m3 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m2.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m4 = M.derived_feature_extractor.v3(
    input_data=m3.data,
    features=m2.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)


[2022-04-26 15:05:22.567566] INFO: moduleinvoker: instruments.v2 开始运行..

[2022-04-26 15:05:22.635856] INFO: moduleinvoker: instruments.v2 运行完成[0.068282s].

[2022-04-26 15:05:22.639995] INFO: moduleinvoker: input_features.v1 开始运行..

[2022-04-26 15:05:22.652439] INFO: moduleinvoker: 命中缓存

[2022-04-26 15:05:22.653972] INFO: moduleinvoker: input_features.v1 运行完成[0.013997s].

...

[2022-04-26 15:05:23.587641] INFO: 基础特征抽取: 总行数: 344895

[2022-04-26 15:05:23.600523] INFO: moduleinvoker: general_feature_extractor.v7 运行完成[0.934162s].

[2022-04-26 15:05:23.608580] INFO: moduleinvoker: derived_feature_extractor.v3 开始运行..

[2022-04-26 15:05:24.858932] INFO: derived_feature_extractor: /y_2022, 344895

[2022-04-26 15:05:24.950585] INFO: moduleinvoker: derived_feature_extractor.v3 运行完成[1.341995s].

In [35]:
# 读取数据
df_total = m4.data.read()
df_total.fillna(0, inplace=True)

# 统计函数
def cal_limit(x):
    x['上涨数'] = len(x[x.return_0 > 1])  # 收益率>1
    x['平家数'] = len(x[x.return_0 == 1])  # 收益率=1
    x['下跌数'] = len(x[(x.return_0 < 1)&(x.return_0 > 0)])  # 收益率<1
    x['停牌'] = len(x[x.return_0 == 0])  # 收益率=0
    x['涨停数'] = len(x[x.price_limit_status_0==3])  # 平台预处理标识
    x['跌停数'] = len(x[x.price_limit_status_0==1])
    return x

df_limit = df_total.groupby('date').apply(cal_limit)
df_limit = df_limit.drop_duplicates(subset=['date'],keep='last').sort_values('date').reset_index()

# 保存在根目录下
df_limit[['date','上涨数','平家数','下跌数','停牌','涨停数','跌停数']].to_csv('/home/bigquant/work/涨跌幅统计.csv')
df_limit

,index,date,instrument,price_limit_status_0,return_0,上涨数,平家数,下跌数,停牌,涨停数,跌停数
0,344815,2022-01-04,872925.BJA,2.0,0.943896,3309,50,1310,0,125,25
1,344816,2022-01-05,872925.BJA,2.0,0.988698,1152,83,3435,1,72,41
2,344817,2022-01-06,872925.BJA,2.0,1.013548,2985,152,1535,2,99,9
3,344818,2022-01-07,872925.BJA,2.0,0.976190,986,70,3619,2,53,28
4,344819,2022-01-10,872925.BJA,2.0,1.026530,3105,144,1428,0,91,17
...,...,...,...,...,...,...,...,...,...,...,...
68,344890,2022-04-19,873169.BJA,2.0,0.988994,2717,133,1912,5,73,44
69,344891,2022-04-20,873169.BJA,2.0,0.974563,1314,76,3377,3,66,43
70,344892,2022-04-21,873169.BJA,2.0,1.008157,335,16,4421,3,42,123
71,344893,2022-04-22,873169.BJA,2.0,1.012945,1647,91,3036,4,86,107


In [36]:
# 绘图
options = {'chart': {'type': 'spline', 'height': 600},}
T.plot(df_limit[['date', '上涨数', '平家数', '下跌数', '停牌']].set_index('date'),title='每日行情走势统计',options=options)
T.plot(df_limit[['date', '涨停数', '跌停数']].set_index('date'),title='每日涨跌停统计',options=options)